In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
import random
import itertools
from IPython.display import clear_output

In [107]:
def handle_table_to_line(mã):
    global df1
    df1= pd.read_excel("./All_prices/%s.xlsx"%(mã))
    # df1.rename(columns={'STT','Ngày','Giá Tham Chiếu','+/- (*)','% (*)','Đóng Cửa (*)','Khối Lượng','Mở Cửa (*)','Cao Nhất (*)','Thấp Nhất (*)','Giao Dịch Thỏa Thuận','Nước Ngoài Mua','Nước Ngoài Bán','Giá Trị(triệu đồng)'})
    df1.columns = [''] * len(df1.columns)
    header = df1.iloc[0].values
    df1.columns = header
    df1.drop(0,inplace=True)
    df1.drop(0,axis=1,inplace=True)
    df = df1[["Đóng Cửa (*)","Ngày"]]
    df.reset_index(inplace=True)
    df["Đóng Cửa (*)"]= df["Đóng Cửa (*)"].astype(float)
    df["Ngày"] = pd.to_datetime(df["Ngày"])
    df.sort_values(by="Ngày",ascending=True,inplace=True)

#     MA250= df["Đóng Cửa (*)"].rolling(250).mean()
    
    fig, ax = plt.subplots()
    fig.set_size_inches(11.7, 8.27)
    sns.lineplot(x='Ngày',y='Đóng Cửa (*)', data = df,ax = ax,sort=False)
    plt.savefig('./Strategy/Check_lonely/%s.png'%(mã))
    return df

In [90]:
def corr(ngành,list_stock):
    global cor_tab
    i=0
    max_len=list_stock[0]
    for dataC in list_stock:
        clear_output()
        i+=1
        print(i)
        A= pd.read_excel("./All_prices/%s.xlsx"%(dataC))
        if len(A)>len(pd.read_excel("./All_prices/%s.xlsx"%(max_len))):
            max_len=dataC
        else:
            pass
    
    print("Tìm xong max_len",max_len)
    
    df_head= pd.read_excel("./All_prices/%s.xlsx"%(max_len))    
    print(1)
    
    header = df_head.iloc[0].values
    df_head.columns = header
    df_head.drop(0,inplace=True)
    df_head.drop(0,axis=1,inplace=True)
    df_head["Ngày"] = pd.to_datetime(df_head["Ngày"],dayfirst=True)
    df_head["Đóng Cửa (*)"]= df_head["Đóng Cửa (*)"].astype(float)
    df_head.sort_values(by="Ngày",ascending=True,inplace=True)
    
    cor_tab= pd.DataFrame((df_head[(df_head["Ngày"]>pd.to_datetime('31-12-2016',dayfirst=True))&(df_head["Ngày"]<= pd.to_datetime('01-11-2019',dayfirst=True))])[["Ngày","Đóng Cửa (*)"]])
    
#     cor_tab.rename(columns={df_head.columns[6]:"A"})
    
    cor_tab.rename(columns={"Đóng Cửa (*)":max_len},inplace=True)
    
    cor_tab.reset_index(drop=True,inplace=True)
    cor_tab.reset_index(inplace=True)
#     print(cor_tab)
    
    list_stock.remove(max_len)
    
    for mã in list_stock:
        df2= pd.read_excel("./All_prices/%s.xlsx"%(mã))
        
        header2 = df2.iloc[0].values
        df2.columns = header2
        df2.drop(0,inplace=True)
        df2.drop(0,axis=1,inplace=True)
        df2["Ngày"] = pd.to_datetime(df2["Ngày"],dayfirst=True)
        df2["Đóng Cửa (*)"]= df2["Đóng Cửa (*)"].astype(float)
        df2.sort_values(by="Ngày",ascending=True,inplace=True)
        df3 = df2[(df2['Ngày'] > pd.to_datetime('31-12-2016',dayfirst=True)) & (df2['Ngày'] <= pd.to_datetime('01-11-2019',dayfirst=True))][["Đóng Cửa (*)","Ngày"]]
#         print(df3)
        df3.drop(columns='Ngày',inplace=True)
        
        if (len(df3)) >= (len(cor_tab)):
            K= df3.reset_index(drop=True)
            M= K.reset_index()
            M.columns=["index",mã]
            cor_tab=cor_tab.merge(M,how='inner',left_on= "index", right_on="index")
            
        else:
            appending=[]
            print(len(cor_tab)-len(df3))
            for i in range(len(cor_tab)-len(df3)):
                appending.append(np.nan)
            C=pd.DataFrame(appending)[0]
            D= C.append(df3)
            D=pd.DataFrame(D)
            D.reset_index(drop=True,inplace=True)
            D.reset_index(inplace=True)
#             print(D)
            D.drop(columns=0,inplace=True)
            D.columns=["index",mã]
        
            cor_tab=cor_tab.merge(D,how='inner',left_on= "index", right_on="index")
            
        clear_output()
        print(list_stock.index(mã)+1,mã)
#     cor_tab.drop(columns='Đóng Cửa (*)',inplace=True)
    cor_tab.drop(columns='index',inplace=True)
    cor_tab.reset_index(drop=True,inplace=True)
    
    cor_tab.dropna(axis=1, how='all',inplace=True)
#     cor_tab.drop(columns='Đóng Cửa (*)',inplace=True)
#   cor_tab.drop(0,inplace=True)
    return cor_tab

In [62]:
data= input("Nhập tên ngành:")
f= open("./%s/%s.txt" %(data,data), "r")
String_tên_mã= f.read()
list_stocks= String_tên_mã.split(",")
f.close()
try:
    list_stocks.remove("")
except:
    pass
print(len(list_stocks),list_stocks)

Nhập tên ngành:All_prices
1548 ['AAM', 'A32', 'AAA', 'AAV', 'ABC', 'ABI', 'ABT', 'AC4', 'ACB', 'ACC', 'ACE', 'ACL', 'ACM', 'ACS', 'ACV', 'ADC', 'ADP', 'ADS', 'AFX', 'AG1', 'AGC', 'AGD', 'AGF', 'AGM', 'AGP', 'AGR', 'AGX', 'ALP', 'ALT', 'ALV', 'AMC', 'AMD', 'AME', 'AMP', 'AMS', 'AMV', 'ANT', 'ANV', 'APC', 'APF', 'APG', 'API', 'APP', 'APS', 'APT', 'ARM', 'ART', 'ASA', 'ASD', 'ASIAGF', 'ASM', 'ASP', 'AST', 'ATA', 'ATB', 'ATD', 'ATG', 'ATS', 'AUM', 'AVC', 'AVF', 'AVS', 'B82', 'BAB', 'BAM', 'BAS', 'BAX', 'BBC', 'BBM', 'BBS', 'BBT', 'BCC', 'BCE', 'BCG', 'BCI', 'BCM', 'BCP', 'BDB', 'BDC', 'BDF', 'BDP', 'BDT', 'BDW', 'BED', 'BFC', 'BGM', 'BGW', 'BHC', 'BHK', 'BHN', 'BHP', 'BHS', 'BHT', 'BHV', 'BIC', 'BID', 'BII', 'BKC', 'BKH', 'BLF', 'BLI', 'BLN', 'BLT', 'BM9', 'BMC', 'BMD', 'BMF', 'BMI', 'BMJ', 'BMP', 'BMS', 'BNW', 'BOT', 'BPC', 'BPW', 'BQB', 'BRC', 'BRR', 'BRS', 'BSA', 'BSC', 'BSD', 'BSG', 'BSH', 'BSI', 'BSL', 'BSP', 'BSQ', 'BSR', 'BST', 'BT1', 'BT6', 'BTB', 'BTC', 'BTD', 'BTG', 'BTH', 'BTN',